In [1]:
import os 
## Because the KERAS used is a old verion, KERAS 1
os.environ['TF_USE_LEGACY_KERAS'] = '1'

## Allows memory allocation on the GPU to be done asynchronously, 
## potentially improving performance and efficiency in certain scenarios.
os.environ['TF_GPU_ALLOCATOR'] = 'cuda_malloc_async'

## Specify which GPUs are available to a program.
## The list of GPUs is usually represented by indices starting at 0.
os.environ['CUDA_VISIBLE_DEVICES'] = '-1' # disable cuda

# Install VideoImageTools

To install VideoImageTools go to next link https://github.com/trucomanx/VideoImageTools

# Install OpenPifPafTools

To install OpenPifPafTools go to next link https://github.com/trucomanx/OpenPifPafTools


In [2]:
input_default_json_conf_file='testing_over_video.json';

# Bibliotecas externas

In [3]:
import platform, sys, os

import json
import VideoImageTools as vit
import OpenPifPafTools.OpenPifPafAnnotations as opp
import OpenPifPafTools.OpenPifPafGetData as oppgd

import cv2
import openpifpaf
from PIL import Image

/home/fernando/.local/lib/python3.10/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [4]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

2024-09-04 16:32:14.525683: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

RuntimeError: module compiled against API version 0x10 but this version of numpy is 0xf

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception

# Biblioteca local

In [ ]:
sys.path.append('../library');

In [ ]:
import SystemEmotion4Lib.Classifier as mylib

# Variables

In [ ]:
## Load json conf file
fd = open(os.path.join('./',input_default_json_conf_file));
DATA = json.load(fd);
fd.close()

In [ ]:
## Model of network
checkpoint = DATA["checkpoint"];
model_type_body = DATA["model_type_body"]; ## 'mobilenet_v3', 'efficientnet_b3', 'inception_v3', 'inception_resnet_v2', 'resnet_v2_50'
model_type_face = DATA["model_type_face"]; ## 'mobilenet_v3', 'efficientnet_b3', 'inception_v3', 'inception_resnet_v2', 'resnet_v2_50'
model_type_skel = DATA["model_type_skel"];
model_type_fusion = DATA["model_type_fusion"];
enable_minus = DATA["enable_minus"];

vin_path = DATA['input_mp4_file'];

output_base_dir = DATA['output_base_dir'];

sub_dir='sub_dir1';

body_factor=1.0;
face_factor=0.85;


# If command line

In [ ]:
for n in range(len(sys.argv)):
    if   sys.argv[n]=='--model-type-check':
        checkpoint=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-body':
        model_type_body=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-face':
        model_type_face=sys.argv[n+1];
    elif sys.argv[n]=='--model-type-skel':
        model_type_skel=int(sys.argv[n+1]);
    elif sys.argv[n]=='--model-type-fusion':
        model_type_fusion=int(sys.argv[n+1]);
    elif sys.argv[n]=='--enable-minus':
        enable_minus=sys.argv[n+1].lower()=='true';
    elif sys.argv[n]=='--input-file':
        vin_path=sys.argv[n+1];
    elif sys.argv[n]=='--output-dir':
        output_base_dir=sys.argv[n+1];
    elif sys.argv[n]=='--sub-dir':
        sub_dir=sys.argv[n+1];


print('')
print('       checkpoint:',checkpoint);
print('  model_type_body:',model_type_body);
print('  model_type_face:',model_type_face);
print('  model_type_skel:',model_type_skel);
print('model_type_fusion:',model_type_fusion);
print('     enable_minus:',enable_minus);
print('         vin_path:',vin_path);
print('  output_base_dir:',output_base_dir);
print('          sub_dir:',sub_dir);
print('      body_factor:',body_factor);
print('      face_factor:',face_factor);
print('')

In [ ]:
vout_dir_path=os.path.join(output_base_dir,sub_dir,'test_over_video');
print('vout_dir_path:',vout_dir_path);

In [ ]:

vout_path=os.path.join(vout_dir_path,os.path.basename(vin_path));
print('vout_path:',vout_path);

# Creating output directory

In [ ]:
os.makedirs(vout_dir_path,exist_ok=True); 
print('Created output_dir:',vout_dir_path);

# Auxiliar functions

In [ ]:
import numpy as np
import tensorflow as tf
import torch

def extra_func(cap):
    # Libera a memória da GPU no PyTorch
    torch.cuda.empty_cache()
    
    # Limpa a sessão atual e libera a memória da GPU no TensorFlow
    tf.keras.backend.clear_session()
    print('clean')

def my_func(Clf,frame):
    #categories=['angry','disgusted','fearful','happy','neutral','sad','surprised'];
    categories=['negative','neutral','pain','positive'];
    
    img_tmp = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB);
    pil_im = Image.fromarray(img_tmp);
    
    res, res_face, res_body, res_skel, face_bbox, body_bbox = Clf.predict_all_pil(pil_im);

    if res is None:
        return frame;
    
    res = np.argmax(res);
    texto=categories[res];
    (xi,yi,xo,yo) = body_bbox;
    color=(0,255,0);
    thickness=2;
    
    frame = cv2.putText(frame,
                        texto,
                        org = (int(xi), int((yi+yo)/2)),
                        fontFace = cv2.FONT_HERSHEY_DUPLEX,
                        fontScale = 2.0,
                        color = (255, 0, 0),
                        thickness = thickness
                        );

    cv2.rectangle(frame,(xi,yi),(xo,yo),color,thickness);
    return frame;




In [ ]:
Clf=mylib.Emotion4Classifier(   checkpoint=checkpoint,
                                model_type_face=model_type_face,
                                model_type_body=model_type_body,
                                model_type_skel=model_type_skel,
                                model_type_skel_enable_minus=enable_minus,
                                model_type_fusion=model_type_fusion,
                                body_factor=body_factor,
                                face_factor=face_factor);

vit.apply_func_predictor_over_video_outmp4(my_func,Clf,vin_path,vout_path,extra_func_counter=1024,extra_func=extra_func)

'''
## Crio pasata com imagens
vout_dir_tmp=os.path.join(vout_dir_path,os.path.splitext(os.path.basename(vin_path))[0]+'_'+model_type);
image_files, fps = vit.apply_func_predictor_over_video_to_frames(my_func,(Clf,predictor),vin_path,vout_dir_tmp, show=False,FORMATO = "frame_{:05d}.png");

## Save frames to video
vit.images_to_video(image_files, fps, vout_path)
'''

print('working end')